# Классификация рукописных цифр

## Загрузка датасета MNIST с Kaggle

In [12]:
!pip install opendatasets

In [13]:
import opendatasets as od

In [14]:
od.download("https://www.kaggle.com/c/digit-recognizer/overview")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading digit-recognizer.zip to ./digit-recognizer


100%|██████████| 15.3M/15.3M [00:00<00:00, 1.22GB/s]


Extracting archive ./digit-recognizer/digit-recognizer.zip to ./digit-recognizer


## Импорт библиотек

In [11]:
import cv2
import numpy as np
import tensorflow as tf 

print(tf.__version__)

2.18.0
